In [77]:
import MetaTrader5 as mt5 
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller 

from datetime import datetime
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'
def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','close']]

def compute_spread(p1, p2, tf, x):
    data1 = get_rates(p1, tf, x)
    data2 = get_rates(p2, tf, x)
    merged = data1.join(data2, lsuffix="_x", rsuffix="_y")
    spread = merged['close_x'] - merged['close_y']
    return spread.dropna()

def adf_test(spread):
    '''Runs ADF test on a spread series'''
    result = adfuller(spread)
    return {'ADF Statistic': result[0], 'p-value': result[1], 'Critical Values': result[4]}

In [78]:
currency_pairs = [
    ('EURUSD.a', 'GBPUSD.a',),
    ('AUDCAD.a', 'USDCAD.a',),
    ('NZDCAD.a', 'USDCAD.a',),
    ('EURCHF.a', 'GBPCHF.a',),
    ('EURNOK.a', 'EURSEK.a',)
]

In [81]:
total_df = pd.DataFrame()  # Initialize total_df before the loop
for currency in currency_pairs:
    df = pd.DataFrame(compute_spread(currency[0], currency[1], mt5.TIMEFRAME_D1, 2000))
    # Rename all columns in df to reflect the currency pair
    df.columns = [f'{currency[0]}-{currency[1]}' for col in df.columns]
    total_df = pd.concat([total_df, df], axis=1)

### VaR Calculations

In [100]:
total_df_ret = total_df.pct_change().dropna()

# Focusing on negative returns for VaR
negative_returns = total_df_ret[total_df_ret < 0].dropna()
negative_returns.describe()

,EURUSD.a-GBPUSD.a,AUDCAD.a-USDCAD.a,NZDCAD.a-USDCAD.a,EURCHF.a-GBPCHF.a,EURNOK.a-EURSEK.a
count,161.000000,161.000000,161.000000,161.000000,161.000000
mean,-0.023950,-0.013821,-0.012997,-0.026690,-0.340117
std,0.023282,0.011729,0.011439,0.023809,1.405744
min,-0.139372,-0.061058,-0.077931,-0.142918,-17.017341
25%,-0.032961,-0.018721,-0.018807,-0.035660,-0.228913
50%,-0.016546,-0.010318,-0.009317,-0.018752,-0.102356
75%,-0.008768,-0.005332,-0.005029,-0.011114,-0.044549
max,-0.000076,-0.000025,-0.000113,-0.000336,-0.000260


In [102]:
for i in total_df_ret.columns:
    print(i)

EURUSD.a-GBPUSD.a
AUDCAD.a-USDCAD.a
NZDCAD.a-USDCAD.a
EURCHF.a-GBPCHF.a
EURNOK.a-EURSEK.a


In [103]:
import numpy as np

var_level = 95
spreads = total_df_ret.columns
var_results = {}

for spread in spreads:
    negative_returns = total_df[spread][total_df[spread] < 0]
    var_95 = np.percentile(negative_returns, 100 - var_level)
    var_95 = abs(var_95)  # Convert to positive number for representation
    var_results[spread] = var_95

In [105]:
var_levels = {}

for col in total_df_ret.columns:
    var_95 = np.percentile(total_df_ret[col].dropna(), 5)  # 5th percentile for 95% VaR
    var_levels[col] = var_95

# Display VaR for each currency pair
for pair, var in var_levels.items():
    print(f"95% VaR for {pair}: {var * 100:.2f}%")

95% VaR for EURUSD.a-GBPUSD.a: -5.54%
95% VaR for AUDCAD.a-USDCAD.a: -3.19%
95% VaR for NZDCAD.a-USDCAD.a: -2.89%
95% VaR for EURCHF.a-GBPCHF.a: -5.70%
95% VaR for EURNOK.a-EURSEK.a: -60.59%


In [104]:
var_results

{'EURUSD.a-GBPUSD.a': 0.2243070000000001,
 'AUDCAD.a-USDCAD.a': 0.48728,
 'NZDCAD.a-USDCAD.a': 0.5537754999999999,
 'EURCHF.a-GBPCHF.a': 0.21131300000000003,
 'EURNOK.a-EURSEK.a': 0.8948379999999997}

In [50]:
total_df.corr()

,EURUSD.a-GBPUSD.a,AUDCAD.a-USDCAD.a,NZDCAD.a-USDCAD.a,EURCHF.a-GBPCHF.a,EURNOK.a-EURSEK.a
EURUSD.a-GBPUSD.a,1.000000,0.241742,0.221649,0.981762,0.105700
AUDCAD.a-USDCAD.a,0.241742,1.000000,0.953249,0.232592,-0.244765
NZDCAD.a-USDCAD.a,0.221649,0.953249,1.000000,0.201966,-0.228926
EURCHF.a-GBPCHF.a,0.981762,0.232592,0.201966,1.000000,0.143348
EURNOK.a-EURSEK.a,0.105700,-0.244765,-0.228926,0.143348,1.000000


### Monte Carlo sims

In [1]:
# Random Walks

### "Swap" risk calculations